In [1]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
OSM = pd.read_json('amenities-vancouver.json.gz', lines=True)
OSM

,lat,lon,timestamp,amenity,name,tags
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ..."
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...
2,49.373423,-123.291894,2016-10-10T02:14:29.000-07:00,toilets,NaN,{}
3,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,NaN,{}
4,49.370898,-123.280448,2015-05-03T00:42:25.000-07:00,place_of_worship,St. Monica's Anglican Church,"{'addr:housenumber': '6404', 'addr:street': 'W..."
...,...,...,...,...,...,...
17713,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{}
17714,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'}
17715,49.276443,-122.790138,2013-03-26T23:45:50.000-07:00,parking,NaN,{}
17716,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne..."


In [2]:
OSM.amenity.unique()

array(['cafe', 'fast_food', 'toilets', 'bbq', 'place_of_worship',
       'post_box', 'telephone', 'fuel', 'vending_machine', 'restaurant',
       'parking_entrance', 'pub', 'bicycle_parking', 'school', 'bench',
       'community_centre', 'waste_basket', 'pharmacy', 'dentist',
       'doctors', 'post_office', 'atm', 'childcare', 'parking',
       'public_building', 'bank', 'cinema', 'theatre', 'ferry_terminal',
       'bar', 'library', 'car_rental', 'drinking_water', 'shelter',
       'car_sharing', 'bicycle_rental', 'clinic', 'recycling',
       'public_bookcase', 'university', 'dojo', 'food_court',
       'seaplane terminal', 'arts_centre', 'ice_cream',
       'charging_station', 'car_wash', 'fountain', 'veterinary',
       'bicycle_repair_station', 'waste_disposal', 'photo_booth',
       'luggage_locker', 'bureau_de_change', 'parking_space', 'nightclub',
       'social_facility', 'taxi', 'bus_station', 'college',
       'construction', 'post_depot', 'nursery', 'clock', 'kindergarten'

In [3]:
rest_list = ['cafe', 'fast_food', 'bbq', 'restaurant', 'pub',
       'bar', 'food_court', 'ice_cream', 'bistro', 'juice_bar',
       'lounge', 'disused:restaurant', 'biergarten']
restaurants = OSM[OSM.amenity.isin(rest_list)]

In [4]:
pd.set_option('display.max_rows', 100)
restaurants[restaurants.name.isnull()]

,lat,lon,timestamp,amenity,name,tags
3,49.249848,-122.959708,2011-09-06T03:52:10.000-07:00,bbq,NaN,{}
296,49.266588,-123.242722,2011-04-11T20:55:49.000-07:00,food_court,NaN,{}
786,49.049771,-122.319001,2019-09-02T22:08:26.000-07:00,fast_food,NaN,"{'official_name': 'Kami Sushi Enterprises', 'a..."
1263,49.237670,-122.782747,2012-03-01T05:24:27.000-08:00,bbq,NaN,{}
1553,49.263266,-123.110529,2014-07-11T19:59:54.000-07:00,pub,NaN,{}
2046,49.229367,-123.004155,2017-03-22T04:32:05.000-07:00,bar,NaN,{'addr:housenumber': '1822'}
2580,49.288112,-123.114637,2017-03-18T01:54:53.000-07:00,cafe,NaN,{}
3609,49.333194,-123.089975,2017-06-27T17:24:54.000-07:00,cafe,NaN,{}
3632,49.184725,-122.798876,2019-02-24T16:44:29.000-08:00,food_court,NaN,{'indoor': 'yes'}
4540,49.289667,-122.785343,2017-07-10T00:38:28.000-07:00,fast_food,NaN,{}


In [5]:
restaurants[~restaurants.name.isnull()]

,lat,lon,timestamp,amenity,name,tags
0,49.260812,-123.125736,2020-03-20T18:22:12.000-07:00,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ..."
1,49.260953,-123.125704,2019-08-02T18:11:20.000-07:00,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...
13,49.126650,-123.182470,2020-03-30T09:08:51.000-07:00,restaurant,Best Bite Indian Cuisine,"{'addr:housenumber': '10-3891', 'phone': '+1-6..."
16,49.283192,-123.109050,2015-12-18T21:41:07.000-08:00,pub,The Cambie,"{'toilets:wheelchair': 'no', 'wheelchair': 'li..."
19,49.265951,-123.246630,2011-11-19T08:06:36.000-08:00,pub,Mahony and Sons,{'wheelchair': 'yes'}
...,...,...,...,...,...,...
17712,49.250408,-123.076261,2017-07-08T05:22:57.000-07:00,restaurant,House of Dosas,"{'addr:housenumber': '1391', 'phone': '+1-604-..."
17713,49.278424,-122.806704,2013-03-26T23:45:49.000-07:00,cafe,Creekside Coffee,{}
17714,49.278770,-122.797628,2013-03-26T23:45:49.000-07:00,restaurant,Togo Sushi,{'cuisine': 'japanese'}
17716,49.282666,-122.826978,2019-09-13T13:56:49.000-07:00,pub,Brown's Social House,"{'addr:housenumber': '215', 'brewery': 'Guinne..."


In [18]:
print(len(restaurants.name.unique()))
restaurants.name.unique()

3481


array(['Starbucks', 'Salad Loop', nan, ..., 'House of Dosas',
       'Creekside Coffee', "Gallagher's Cafe"], dtype=object)